In [9]:
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import multivariate_normal

def fit(Y,K,p,labels,max_iter):
    #p is the degree of the polynomial regression
    #K is the number of classes
    #Y is a n*d obeservation matrix
    #labels is the labels matrix of Y
    """ Initialisation """
    n,d = Y.shape
    pf = PolynomialFeatures(degree=p) #transformer to get polynomial features of the time
    t = pf.fit_transform(np.arange(n).reshape(-1,1)) #matrix containing the polynomial features of the time
    pi = (1.0/K)*np.ones((K))
    A = (1.0/(2.0*(K-1)))*np.ones((K,K))
    for i in range(K):
        A[i,i] = 0.5
    B = [] #list of coefficient matrices
    sigma = [] #list of covariance matrices 
    for i in range(K):
        B.append(np.random.uniform(-1,1,(p+1,d)))
        s = np.transpose(Y[labels==i,:])
        sigma.append(np.cov(s))
    F = np.zeros((n,K))
    for i in range(n):
        for j in range(K):
            mean = np.dot(B[j].T,t[j,:])
            F[i,j] = multivariate_normal(mean=mean,cov=sigma[j],allow_singular=True).pdf(Y[i,:])
    for i in range(max_iter):
        """ E step """
        #appeler tranpose(F) :c
        #calculate P1 and P2
        #P1 is a n*K matrix
        #P2 is a matrix of size n*K*K
        P1 = np.zeros((n,K))
        P2 = np.zeros((n,K,K))
        """ M step """
        pi = P1[0,:]
        A = np.transpose(np.sum(P2[1:,:,:],axis=0))
        for i in range(K):
            A[i,:] = A[i,:]/np.sum(P1[1:,:],axis=0)
            W = np.diag(P1[:,i])
            M = np.dot(np.transpose(t),np.dot(W,t))
            B[i] = np.dot(np.linalg.inv(M),np.transpose(t))
            B[i] = np.dot(B[i],np.dot(W,Y))
            c = 1.0/np.sum(P1[:,k])
            M = Y-np.dot(t,B[i])
            sigma[i] = c*np.dot(np.transpose(M),np.dot(W,M))
        #calculate the Bs and the sigmas
        """ Update F """
        for i in range(n):
            for j in range(K):
                mean = np.dot(B[j].T,t[j,:])
                F[i,j] = multivariate_normal(mean=mean,cov=sigma[j]).pdf(Y[i,:])
    return pi,A,B,sigma

In [3]:
import numpy as np
A = np.zeros((2,3))

In [3]:
a,b = A.shape

In [6]:
b

3

In [20]:
np.cov(np.transpose(A))

array([[ 0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5]])

In [19]:
def fwd_bkw(int_dist,F,A):
    

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  1.]])

In [18]:
A[1,2] = 1

In [23]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=3)

In [28]:
pf.fit_transform(np.arange(3).reshape(-1,1))

array([[ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  2.,  4.,  8.]])

In [37]:
from scipy.stats import multivariate_normal

0.3989422804014327

In [52]:
a = np.ones((3,4,2))

In [53]:
a.shape

(3, 4, 2)

In [54]:
np.sum(a,axis=0).shape

(4, 2)

In [58]:
print(np.arange(2,5))

[2 3 4]


In [61]:
np.linalg.inv(np.diag(np.ones(3)))

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [10]:
fit(np.ones((100,25)),3,5,np.random.randint(0,4,size=100),100)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: RuntimeWarning: invalid value encountered in true_divide


LinAlgError: Singular matrix

In [77]:
Y = np.ones((100,25))
labels = np.ones(100)

In [90]:
np.random.randint(0,4,size=100)

array([3, 2, 2, 0, 2, 0, 1, 2, 3, 1, 3, 2, 0, 0, 3, 0, 1, 3, 1, 3, 1, 2, 3,
       3, 0, 3, 3, 2, 3, 3, 0, 0, 2, 2, 3, 0, 3, 3, 1, 0, 2, 1, 3, 0, 2, 3,
       3, 0, 3, 1, 2, 0, 1, 1, 0, 1, 3, 2, 3, 0, 0, 0, 3, 0, 3, 0, 3, 2, 0,
       0, 3, 2, 1, 0, 2, 3, 0, 2, 0, 3, 1, 2, 0, 0, 3, 1, 1, 3, 1, 3, 3, 1,
       3, 1, 2, 0, 1, 2, 1, 0])